In [1]:
import requests
import re
import json
import pandas as pd
import time
from urllib.parse import urlparse
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import os

In [2]:
# 1. Obtener el HTML de la página del ranking
load_dotenv()
url = os.getenv("TARGET_URL_PRIVATE_CREDIT")
BASE_URL = os.getenv("BASE_URL")

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36"
}

response = requests.get(url, headers=headers)
response.raise_for_status()  # Lanza error si la petición falla

In [3]:
# 2. Parsear HTML con BeautifulSoup
soup = BeautifulSoup(response.text, "html.parser")

In [4]:
# 3. Buscar la tabla principal y extraer los links
table = soup.find("table")
country_links = []


if table:
    rows = table.find_all("tr")
    for row in rows:
        cell = row.find("td")
        if cell and cell.a:
            relative_link = cell.a["href"]
            full_url = f"{BASE_URL}{relative_link}"
            country_links.append(full_url)

# Convertir la lista en DataFrame
df_links = pd.DataFrame(country_links, columns=["url"])

# Guardar como archivo Parquet
df_links.to_parquet("data/country_private_creit_links.parquet", index=False)

In [5]:
# 4. Mostrar resultado
print("Links encontrados:", df_links.size)

Links encontrados: 171


In [6]:
# configuracion necesaria para descargar datos

master_df = pd.DataFrame()

pattern = r'arrayToDataTable\((\[\[.*?\]\])\)'

sleep_duration = 5

In [7]:
# Cargar los países válidos desde el archivo parquet
df_countries = pd.read_parquet("data/countries_with_hpi.parquet")
valid_countries = set(df_countries["country"].unique())

In [8]:
start_time = time.time()
for url in country_links:
    # Extraer país desde el URL
    country = urlparse(url).path.strip("/").split("/")[0]

    # Verificar si el país está en la lista válida
    if country not in valid_countries:
        print(f"⏭️ País {country} no está en la lista, se omite.")
        continue

    try:
        response = requests.get(url, headers=headers)
        html = response.text

        match = re.search(pattern, html, re.DOTALL)
        if match:
            data_str = match.group(1)
            data = json.loads(data_str)

            df = pd.DataFrame(data[1:], columns=data[0])
            df.columns = ["Period", "Value"]
            df["Country"] = country

            df_pivot = df.pivot(index="Period", columns="Country", values="Value")
            master_df = pd.merge(master_df, df_pivot, how="outer", left_index=True, right_index=True)

            print(f"✅ Datos extraídos correctamente para {country}")
        else:
            print(f"⚠️ No se encontraron datos en {url}")
    except Exception as e:
        print(f"❌ Error procesando {url}: {e}")
    
    time.sleep(sleep_duration)

end_time = time.time()
total_sleep_time = len(valid_countries) * sleep_duration
effective_time = end_time - start_time - total_sleep_time

print(f"⏳ Tiempo total (incluyendo esperas): {end_time - start_time:.2f} s")
print(f"😴 Tiempo en espera: {total_sleep_time:.2f} s")
print(f"⚡ Tiempo efectivo de ejecución: {effective_time:.2f} s")

⏭️ País Albania no está en la lista, se omite.
⏭️ País Algeria no está en la lista, se omite.
⏭️ País Angola no está en la lista, se omite.
⏭️ País Antigua-and-Barbuda no está en la lista, se omite.
⏭️ País Argentina no está en la lista, se omite.
⏭️ País Armenia no está en la lista, se omite.
⏭️ País Aruba no está en la lista, se omite.
✅ Datos extraídos correctamente para Australia
✅ Datos extraídos correctamente para Austria
⏭️ País Azerbaijan no está en la lista, se omite.
⏭️ País Bahamas no está en la lista, se omite.
⏭️ País Bahrain no está en la lista, se omite.
⏭️ País Bangladesh no está en la lista, se omite.
⏭️ País Barbados no está en la lista, se omite.
⏭️ País Belarus no está en la lista, se omite.
✅ Datos extraídos correctamente para Belgium
⏭️ País Belize no está en la lista, se omite.
⏭️ País Benin no está en la lista, se omite.
⏭️ País Bolivia no está en la lista, se omite.
⏭️ País Bosnia-and-Herzegovina no está en la lista, se omite.
⏭️ País Botswana no está en la lis

In [9]:
master_df.head()

,Australia,Austria,Belgium,Brazil,Bulgaria,Canada,Chile,China,Colombia,Croatia,...,Slovenia,South-Africa,South-Korea,Spain,Sweden,Switzerland,Thailand,Turkey,USA,United-Kingdom
Period,,,,,,,,,,,,,,,,,,,,,
Apr 1965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Apr 1966,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.77,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Apr 1967,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Apr 1968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Apr 1969,NaN,NaN,NaN,NaN,NaN,76.62,NaN,NaN,NaN,NaN,...,NaN,4.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# obtain paraguay url
template_url = country_links[0]
country = "Paraguay"

# Separar por "/"
parts = template_url.strip("/").split("/")

# Reemplazar el país (posición 3: después del dominio)

parts[3] = country

# Reconstruir la URL
custom_url = "/".join(parts) + "/"

In [11]:
# URL específica para Paraguay
response = requests.get(custom_url, headers=headers)
html = response.text

# Buscar el bloque JSON dentro del HTML
match = re.search(pattern, html, re.DOTALL)
if match:
    data_str = match.group(1)
    data = json.loads(data_str)

    # Crear DataFrame con los datos
    df = pd.DataFrame(data[1:], columns=data[0])
    df.columns = ["Period", "Value"]
    df["Country"] = "Paraguay"  # Asignar explícitamente

    # Pivotear para formato ancho (opcional)
    df_paraguay = df.pivot(index="Period", columns="Country", values="Value")

    print("✅ Datos de Paraguay extraídos correctamente")
else:
    print("⚠️ No se encontraron datos en el HTML para Paraguay")

✅ Datos de Paraguay extraídos correctamente


In [12]:
df_paraguay.head()

Country,Paraguay
Period,
Apr 1996,4340.15
Apr 1997,5428.67
Apr 1998,6672.60
Apr 1999,6051.41
Apr 2000,6630.16


In [13]:
# Unir ambos DataFrames por el índice 'Period', preservando todos los datos
master_df = pd.merge(master_df, df_paraguay, how="outer", left_index=True, right_index=True)

In [14]:
master_df.head()

,Australia,Austria,Belgium,Brazil,Bulgaria,Canada,Chile,China,Colombia,Croatia,...,South-Africa,South-Korea,Spain,Sweden,Switzerland,Thailand,Turkey,USA,United-Kingdom,Paraguay
Period,,,,,,,,,,,,,,,,,,,,,
Apr 1965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Apr 1966,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.77,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Apr 1967,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Apr 1968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Apr 1969,NaN,NaN,NaN,NaN,NaN,76.62,NaN,NaN,NaN,NaN,...,4.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# Resetear el índice para separar "Period"
master_df = master_df.reset_index()

# Mapeo de mes a trimestre
month_to_quarter = {
    "Jan": "Q1", "Feb": "Q1", "Mar": "Q1",
    "Apr": "Q2", "May": "Q2", "Jun": "Q2",
    "Jul": "Q3", "Aug": "Q3", "Sep": "Q3",
    "Oct": "Q4", "Nov": "Q4", "Dec": "Q4"
}

# Extraer mes y año desde "Period"
master_df[["Month", "Year"]] = master_df["Period"].str.extract(r"(\w{3})\s+(\d{4})")

# Filtrar filas válidas (donde se extrajo correctamente el año)
master_df = master_df[master_df["Year"].notna()].copy()

# Mapear trimestre y convertir año
master_df["Quarter"] = master_df["Month"].map(month_to_quarter)
master_df["Year"] = master_df["Year"].astype(int)

# Reordenar las columnas: primero Quarter y Year
cols = ["Quarter", "Year"] + [col for col in master_df.columns if col not in ["Period", "Month", "Quarter", "Year"]]
master_df = master_df[cols]

# Ordenar por Year y Quarter
master_df = master_df.sort_values(by=["Year", "Quarter"]).reset_index(drop=True)

In [16]:
master_df.head()

,Quarter,Year,Australia,Austria,Belgium,Brazil,Bulgaria,Canada,Chile,China,...,South-Africa,South-Korea,Spain,Sweden,Switzerland,Thailand,Turkey,USA,United-Kingdom,Paraguay
0,Q1,1965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Q2,1965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Q2,1965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Q3,1965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Q3,1965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# Guardar como archivo Parquet
master_df.to_parquet("data/private_credit_values.parquet", index=False)